In [5]:
import pandas as pd

# ----------------------------
# CONFIG (edit paths + formats)
# ----------------------------
BUS_PATH = "/Users/tenzinchoedhen/Desktop/PABT_Bus_Cleaned.csv"
PASS_PATH = "/Users/tenzinchoedhen/Desktop/PABT_Passenger_Cleaned.csv"
OUTPUT_PATH = "/Users/tenzinchoedhen/Desktop/PABT_Weekly_Fact.csv"

DATE_COLS = ["Start_Date", "End_Date"]
CARRIER_COL = "Carrier"
VOLUME_COL = "Volume"

# ----------------------------
# LOAD
# ----------------------------
bus = pd.read_csv(BUS_PATH)
pas = pd.read_csv(PASS_PATH)

# ----------------------------
# CLEAN + STANDARDIZE
# ----------------------------
def standardize(df: pd.DataFrame, volume_name: str) -> pd.DataFrame:
    df = df.copy()

    # normalize column names
    df.columns = [c.strip().replace(" ", "_") for c in df.columns]

    # parse dates
    for c in DATE_COLS:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors="coerce")

    # clean carrier
    if CARRIER_COL in df.columns:
        df[CARRIER_COL] = (
            df[CARRIER_COL]
            .astype(str)
            .str.strip()
            .str.replace(r"\s+", " ", regex=True)
        )

    # ensure volume numeric
    if VOLUME_COL in df.columns:
        df[VOLUME_COL] = (
            df[VOLUME_COL]
            .astype(str)
            .str.replace(",", "", regex=False)
            .str.strip()
        )
        df[VOLUME_COL] = pd.to_numeric(df[VOLUME_COL], errors="coerce")

    # keep required columns
    keep = [c for c in DATE_COLS if c in df.columns] + [CARRIER_COL, VOLUME_COL]
    df = df[keep]

    # rename volume
    df = df.rename(columns={VOLUME_COL: volume_name})

    # aggregate
    group_cols = [c for c in DATE_COLS if c in df.columns] + [CARRIER_COL]
    df = df.groupby(group_cols, dropna=False, as_index=False)[volume_name].sum()

    return df


bus_std = standardize(bus, "Bus_Volume")
pas_std = standardize(pas, "Passenger_Volume")

# ----------------------------
# MERGE
# ----------------------------
join_cols = [c for c in DATE_COLS if c in bus_std.columns and c in pas_std.columns] + [CARRIER_COL]

pabt_weekly = bus_std.merge(
    pas_std,
    on=join_cols,
    how="outer",
    validate="m:m"
)

# ----------------------------
# ADD Month_Year in m/d/yyyy format
# ----------------------------
if "Start_Date" in pabt_weekly.columns:
    pabt_weekly["Month_Year"] = (
        pabt_weekly["Start_Date"]
        .dt.to_period("M")
        .dt.to_timestamp()
        .dt.strftime("%-m/%-d/%Y")   # Linux/Mac
    )

# ----------------------------
# OUTPUT
# ----------------------------
preferred_order = ["Start_Date", "End_Date", "Month_Year", "Carrier", "Bus_Volume", "Passenger_Volume"]
cols = [c for c in preferred_order if c in pabt_weekly.columns] + [c for c in pabt_weekly.columns if c not in preferred_order]

pabt_weekly = pabt_weekly[cols].sort_values(
    [c for c in ["Start_Date", "Carrier"] if c in pabt_weekly.columns]
)

pabt_weekly.to_csv(OUTPUT_PATH, index=False)

print("✅ Saved:", OUTPUT_PATH)
print("Rows:", len(pabt_weekly))
print(pabt_weekly.head(10))


✅ Saved: /Users/tenzinchoedhen/Desktop/PABT_Weekly_Fact.csv
Rows: 2797
   Start_Date   End_Date Month_Year            Carrier  Bus_Volume  \
1  2020-12-07 2020-12-11  12/1/2020            ACADEMY        37.0   
2  2020-12-07 2020-12-11  12/1/2020      C&J BUS LINES         1.0   
3  2020-12-07 2020-12-11  12/1/2020          COACH USA        83.0   
4  2020-12-07 2020-12-11  12/1/2020             DECAMP         0.0   
5  2020-12-07 2020-12-11  12/1/2020          GREYHOUND        33.0   
6  2020-12-07 2020-12-11  12/1/2020   HCEE - COMMUNITY        74.0   
7  2020-12-07 2020-12-11  12/1/2020           LAKELAND        13.0   
8  2020-12-07 2020-12-11  12/1/2020              MARTZ        14.0   
9  2020-12-07 2020-12-11  12/1/2020         NJ TRANSIT      2199.0   
10 2020-12-07 2020-12-11  12/1/2020  PETER PAN_BONANZA        12.0   

    Passenger_Volume  
1              613.0  
2                8.0  
3             1605.0  
4                0.0  
5              702.0  
6             1199.0

In [6]:
import pandas as pd

INPUT_PATH = "/Users/tenzinchoedhen/Desktop/PABT_Weekly_Fact.csv"
OUTPUT_PATH = "/Users/tenzinchoedhen/Desktop/PABT_Monthly_Fact.csv"

df = pd.read_csv(INPUT_PATH)

# Ensure Month_Year exists (and normalize if needed)
if "Month_Year" not in df.columns:
    df["Start_Date"] = pd.to_datetime(df["Start_Date"], errors="coerce")
    df["Month_Year"] = (
        df["Start_Date"]
        .dt.to_period("M")
        .dt.to_timestamp()
        .dt.strftime("%-m/%-d/%Y")   # Mac/Linux
    )

# Make sure numeric fields are numeric
for col in ["Bus_Volume", "Passenger_Volume"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Monthly aggregation
monthly_fact = (
    df.groupby(["Month_Year", "Carrier"], as_index=False)
      .agg({
          "Bus_Volume": "sum",
          "Passenger_Volume": "sum"
      })
)

# Optional: sort nicely (convert Month_Year back to datetime for sorting)
monthly_fact["_month_sort"] = pd.to_datetime(monthly_fact["Month_Year"], errors="coerce")
monthly_fact = monthly_fact.sort_values(["_month_sort", "Carrier"]).drop(columns=["_month_sort"])

monthly_fact.to_csv(OUTPUT_PATH, index=False)

print("✅ Saved:", OUTPUT_PATH)
print("Rows:", len(monthly_fact))
print(monthly_fact.head(10))


✅ Saved: /Users/tenzinchoedhen/Desktop/PABT_Monthly_Fact.csv
Rows: 648
    Month_Year            Carrier  Bus_Volume  Passenger_Volume
156  12/1/2020            ACADEMY       105.0            1762.0
157  12/1/2020      C&J BUS LINES         4.0              28.0
158  12/1/2020          COACH USA       329.0            6000.0
159  12/1/2020             DECAMP         0.0               0.0
160  12/1/2020          GREYHOUND       143.0            3522.0
161  12/1/2020   HCEE - COMMUNITY       218.0            3524.0
162  12/1/2020           LAKELAND        50.0            1038.0
163  12/1/2020              MARTZ        53.0            1857.0
164  12/1/2020         NJ TRANSIT      8364.0          101072.0
165  12/1/2020  PETER PAN_BONANZA        40.0            1014.0


In [14]:
import pandas as pd

# ----------------------------
# CONFIG (edit these paths)
# ----------------------------
TRAFFIC_PATH = "/Users/tenzinchoedhen/Desktop/PABT_Traffic_Sampled_Cleaned.csv"          # or .txt
SPEED_PATH   = "/Users/tenzinchoedhen/Desktop/Mobility_Speeds_Cleaned.csv"      # or .txt
OUTPUT_PATH  = "/Users/tenzinchoedhen/Desktop/PABT_Traffic_with_Mobility_Speeds.csv"

# If your inputs are .txt, set delimiter (tab is common)
TRAFFIC_IS_TXT = False
SPEED_IS_TXT   = False
TRAFFIC_DELIM  = "\t"
SPEED_DELIM    = "\t"

# ----------------------------
# LOAD
# ----------------------------
if TRAFFIC_IS_TXT:
    traffic_df = pd.read_csv(TRAFFIC_PATH, delimiter=TRAFFIC_DELIM)
else:
    traffic_df = pd.read_csv(TRAFFIC_PATH)

if SPEED_IS_TXT:
    speed_df = pd.read_csv(SPEED_PATH, delimiter=SPEED_DELIM)
else:
    speed_df = pd.read_csv(SPEED_PATH)

# ----------------------------
# CLEAN / STANDARDIZE COLUMNS
# ----------------------------
# Strip whitespace from column names
traffic_df.columns = traffic_df.columns.str.strip()
speed_df.columns = speed_df.columns.str.strip()

# ---- Ensure numeric join keys in traffic ----
# traffic keys: Fac, Month, YR
for col in ["FAC", "Month", "Yr"]:
    if col in traffic_df.columns:
        traffic_df[col] = pd.to_numeric(traffic_df[col], errors="coerce")

# ---- Parse Month_Year in speed ----
# speed keys: Facility_Order, Month_Year -> Month, YR
if "Month_Year" not in speed_df.columns:
    raise ValueError("Speed table must contain 'Month_Year' column.")

# Try robust parsing for Month_Year (handles '2025-01', 'Jan-2025', '01/2025', etc.)
speed_df["Month_Year_parsed"] = pd.to_datetime(speed_df["Month_Year"], errors="coerce")

# If parsing failed for many rows, try common formats explicitly
if speed_df["Month_Year_parsed"].isna().mean() > 0.2:
    # Try MM/YYYY
    speed_df["Month_Year_parsed"] = pd.to_datetime(speed_df["Month_Year"], format="%m/%Y", errors="coerce")

if speed_df["Month_Year_parsed"].isna().mean() > 0.2:
    # Try Mon-YYYY
    speed_df["Month_Year_parsed"] = pd.to_datetime(speed_df["Month_Year"], format="%b-%Y", errors="coerce")

# Create Month and YR from parsed date
speed_df["Month"] = speed_df["Month_Year_parsed"].dt.month
speed_df["YR"] = speed_df["Month_Year_parsed"].dt.year

# Ensure Facility_Order is numeric (since you said Fac == Facility_Order)
if "Facility_Order" not in speed_df.columns:
    raise ValueError("Speed table must contain 'Facility_Order' column.")

speed_df["Facility_Order"] = pd.to_numeric(speed_df["Facility_Order"], errors="coerce")

# Drop speed rows where we can't build join keys (optional but recommended)
speed_df_clean = speed_df.dropna(subset=["Facility_Order", "Month", "YR"]).copy()

# Optional: If mobility table has duplicates per (Facility_Order, Month, YR),
# pick the first (or you can aggregate)
dup_mask = speed_df_clean.duplicated(subset=["Facility_Order", "Month", "YR"], keep=False)
if dup_mask.any():
    # Keep the first occurrence (safe default)
    speed_df_clean = speed_df_clean.sort_values(by=["Facility_Order", "YR", "Month"]).drop_duplicates(
        subset=["Facility_Order", "Month", "YR"],
        keep="first"
    )

# ----------------------------
# LEFT JOIN (keeps ALL traffic records)
# ----------------------------
merged_df = traffic_df.merge(
    speed_df_clean,
    left_on=["FAC", "Month", "Yr"],
    right_on=["Facility_Order", "Month", "YR"],
    how="left",
    suffixes=("", "_speed")
)

# ----------------------------
# QUICK VALIDATION
# ----------------------------
print("Traffic rows:", len(traffic_df))
print("Merged rows :", len(merged_df))

# How many traffic rows found a matching speed row?
# (Avg_Speed might be named differently; adjust if needed.)
speed_match_col = "Avg_Speed" if "Avg_Speed" in merged_df.columns else None
if speed_match_col:
    match_rate = merged_df[speed_match_col].notna().mean()
    print(f"Speed match rate: {match_rate:.2%}")
else:
    # fallback: check if Month_Year_parsed came through
    match_rate = merged_df["Month_Year_parsed"].notna().mean() if "Month_Year_parsed" in merged_df.columns else None
    if match_rate is not None:
        print(f"Speed match rate (by Month_Year_parsed): {match_rate:.2%}")
    else:
        print("Could not compute match rate (no obvious speed columns found).")

# ----------------------------
# SAVE
# ----------------------------
merged_df.to_csv(OUTPUT_PATH, index=False)
print("Saved:", OUTPUT_PATH)


Traffic rows: 104857
Merged rows : 104857
Speed match rate: 9.11%
Saved: /Users/tenzinchoedhen/Desktop/PABT_Traffic_with_Mobility_Speeds.csv


In [17]:
import pandas as pd

# File paths
TRAFFIC_PATH = "/Users/tenzinchoedhen/Desktop/PABT_Traffic_with_Mobility_Speeds.csv"
WEATHER_PATH = "/Users/tenzinchoedhen/Desktop/Weather_Events_Games_Data.csv"
OUTPUT_PATH = "/Users/tenzinchoedhen/Desktop/Traffic_Speed_External_Data.csv.csv"

# -------------------------
# Load both CSVs
# -------------------------
traffic_df = pd.read_csv(TRAFFIC_PATH)
weather_df = pd.read_csv(WEATHER_PATH)

# -------------------------
# Clean column names
# -------------------------
traffic_df.columns = traffic_df.columns.str.strip()
weather_df.columns = weather_df.columns.str.strip()

# -------------------------
# Standardize DATE format
# -------------------------
traffic_df["DATE"] = pd.to_datetime(traffic_df["DATE"], errors="coerce")
weather_df["DATE"] = pd.to_datetime(weather_df["DATE"], errors="coerce")



# -------------------------
# Perform LEFT JOIN
# -------------------------
merged_df = traffic_df.merge(
    weather_df,
    on=["DATE", "TIME"],
    how="left",
    suffixes=("", "_weather")
)

# -------------------------
# Validation
# -------------------------
print("Traffic rows:", len(traffic_df))
print("Merged rows :", len(merged_df))

# Save result
merged_df.to_csv(OUTPUT_PATH, index=False)

print("LEFT JOIN completed successfully.")


/var/folders/_z/yxlqx0wn2l7fv2p0ltxqh0r40000gn/T/ipykernel_73440/3183238373.py:11: DtypeWarning: Columns (31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  traffic_df = pd.read_csv(TRAFFIC_PATH)
/var/folders/_z/yxlqx0wn2l7fv2p0ltxqh0r40000gn/T/ipykernel_73440/3183238373.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  traffic_df["DATE"] = pd.to_datetime(traffic_df["DATE"], errors="coerce")


Traffic rows: 104857
Merged rows : 104857
LEFT JOIN completed successfully.
